<a href="https://colab.research.google.com/github/InsightofSPb/Automatic-word-processing-and-processing-of-images/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Скачиваем классический набор данных -- набор текстов об аэронавтике CRANFIELD

In [111]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


Берём только сами запросы (это будут наши документы)

In [112]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


Объединяем многострочные в один

In [113]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2] #Выведем пару документов для примера

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

Составим запросы к нашим документам

In [114]:
QUERIES = ['electronic computer', 'surface heat']

Boolean retrieval
Представим каждый документ как "битовую маску": вектор размером со словарь, в котором на каждой позиции единица, если в документе есть соответсвующий терм, и ноль, если терма нет

In [115]:
# в разных версиях ответы могут отличаться, поэтому важно иметь одну и ту же
! pip install -q scikit-learn==0.22.2.post1

from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

In [116]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

**Задание 0**

Теперь для каждого из данных запросов QUERIES найдём ближайший для него документ из query_data по сходству Жаккара. Есть более эффективные способы это сделать, но вам требуется реализовать расстояние Жаккара и далее применить его к нашим данным.

In [117]:
import numpy as np 
from sklearn.metrics import jaccard_score
def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Сходство или коэффициент Жаккара: отношение мощности пересечения
    к мощности объединения
  """
  ins = np.logical_and(vector_a, vector_b)
  uni = np.logical_or(vector_a,vector_b)
  simil = ins.sum() / float(uni.sum())
  return simil
#Проверка, что функция работает правильно
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4
print(jaccard_sim(np.array([1, 1, 1, 0, 1, 1, 1, 1, 1]), np.array([1, 1, 1, 1, 1, 0, 0 , 1, 0])))

0.5555555555555556


Теперь для наших запросов рассчитаем самые близкие документы

In [118]:
for q_id, query in enumerate(encoded_queries):
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # вычисляем коэфф. Жаккара
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  # выводим по 3 наиболее близких документа для каждого запроса
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: electronic computer
FOUND:
    15	0.12	can the transverse potential flow about a body of revolution be calculated efficiently by an electronic computer . 
    128	0.08	has anyone programmed a pump design method for a high-speed digital computer . 
    0	0.00	what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . 
Q: surface heat
FOUND:
    45	0.14	what is the combined effect of surface heat and mass transfer on hypersonic flow . 
    8	0.11	papers on internal /slip flow/ heat transfer studies . 
    94	0.10	what is the theoretical heat transfer distribution around a hemisphere . 


**VSM**

Попробуем теперь сделать то же, но с tf-idf и косинусным расстоянием. Мы сделаем всё опять "руками", но "в реальной жизни" лучше использоватьесть эффективные реализации cosine distance, например, из библиотеки scipy.

In [119]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Совет: обязательно разберитесь с тем, какие возможности
# предоставляет tf-idf vectorizer, какие параметры за что отвечают

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

**Задание 2**

Реализовать косинусное расстояние

In [120]:
from numpy import linalg, dot

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Косинусное расстояние: единица минус отношение скалярного произведения
    на произведение L2-норм (подсказка: в numpy такие нормы есть)
  """
  return  1 - dot(vector_a,vector_b) / (linalg.norm(vector_a) * linalg.norm(vector_b))
#Проверка, что функция работает правильно
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5


Теперь вычислим ближайшие по косинусному расстоянию между векторными представлениями документов и запросов

In [123]:
for q_id, query in enumerate(tfidf_encoded_queries):
  
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Косинусное расстояние
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: electronic computer
FOUND:
    15	0.53	can the transverse potential flow about a body of revolution be calculated efficiently by an electronic computer . 
    128	0.76	has anyone programmed a pump design method for a high-speed digital computer . 
    0	1.00	what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . 
Q: surface heat
FOUND:
    45	0.56	what is the combined effect of surface heat and mass transfer on hypersonic flow . 
    44	0.76	has anyone investigated the effect of surface mass transfer on hypersonic viscous interactions . 
    127	0.76	is it possible to obtain a reasonably simple analytical solution to the heat equation for an exponential (in time) heat input . 


<ipython-input-120-dbec1cae7a5c>:8: RuntimeWarning: invalid value encountered in double_scalars
  return  1 - dot(vector_a,vector_b) / (linalg.norm(vector_a) * linalg.norm(vector_b))
